In [ ]:
# Author: bfut <https://github.com/bfut> (zlib License)

# Description

This notebook is a step-by-step converter for FCE files between FCE3, FCE4, and FCE4M formats.

Handles renaming parts, merging parts, reordering parts, renaming dummies, scaling model size, etc., where applicable.

Also converts texture alpha channel. For FCE3 and FCE4, the source is a TGA file. For FCE4M, the source is an FSH file.

Conversion flows: 
```
    FCE4M -> FCE4 -> FCE3
    FCE3  -> FCE4 -> FCE4M

    Ex: an FCE4M file can be converted to FCE4 and FCE3
        an FCE4 file can be converted to FCE4M and FCE3
        an FCE3 file can be converted to FCE4 and FCE4M
```
The input file format is automatically determined. The target format is set as a parameter. See *Parameterize Notebook* for the how-to.

# Purpose

This notebook can produce a game-ready and finished VIV archive in seconds, avoiding manual chores.

Preview and prepare your work for multiple games at the same time.

# Required tools

`Python 3.8 or later`: Windows or Linux, either is possible. *Note:* Windows users may want to consider using WSL

`fcecodec`: https://github.com/bfut/fcecodec (install Python module)

`unvivtool`: https://github.com/bfut/unvivtool (install Python module)

`bfut_NfsTgaConverter.py`: https://github.com/bfut/PyScripts (download)

`bfut_TextureRotator.py`: https://github.com/bfut/PyScripts (download)

`bfut_Tga2Bmp.py`: https://github.com/bfut/PyScripts (download)

`ImageMagick`: https://imagemagick.org/script/download.php (executable) *Note:* only required when working with FSH files

`fshtool v1.22`: https://klein.mit.edu/~auroux/software/fshtool.zip (executable) *Note:* only required when working with FSH files

# Workload

### Parameterize Notebook

Tools
1. `py_scriptsdir` path to a local copy of the `bfut/PyScripts` repository
1. `fcecodec_dir` path to a local copy of the `bfut/fcecodec` repository
1. `fshtool` path to an `fshtool` executable
1. `unvivtool_script` path to a local copy of `unvivtool_script.py` from the `bfut/unvivtool` repository

Files
1. `fce_path_input` path to the source FCE file (can be FCE3, FCE4, or FCE4M)
1. `fce_path_output` will be path to the output FCE file
1. `fsh_path_input` path to an optional source texture in FSH format. Only used, when source file is in FCE4M format.
1. `car00tga_input` path to an optional source texture in TGA format. Only used, when source file is not in FCE4M format.
1. `viv_source_dir` path to the directory whose files will be encoded to the finished VIV archive. Output FCE and texture file should be written to this directory.
1. `fce_path_colorsource` path to an optional source FCE file whose colors will be copied to the output FCE file, if desired.

Misc
1. `copy_color` copy colors from another FCE file (`True` or `False`)
1. `flip_vertically` sometimes a texture has to be flipped vertically (`True` or `False`)
1. `fce_version_target` determines the output FCE format. Set to `3`, `4` or `5` for FCE3, FCE4, or FCE4M, respectively.

In [ ]:
import pathlib

import fcecodec

py_scriptsdir = f"./PyScripts/"
fcecodec_dir = f"./fcecodec/"
fshtool = f"./fshtool"
unvivtool = f"./unvivtool_script.py"

fce_path_input = f"car.fce"
fce_path_output = f"car.fce"

fsh_path_input = f"file.fsh"

car00tga_input = f"car00.tga"
car00tga_output = f"car00.tga"

viv_source_dir = f"car_viv"

fce_path_colorsource = f"car.fce"

copy_color = True
flip_vertically = True

fce_version_target = 3

In [ ]:
if fce_version_target in ["4M", "4m"]:
    fce_version_target = 5
fce_version_target = int(fce_version_target)
assert fce_version_target in [3, 4, 5]

def GetFceVersion(path):
    with open(path, "rb") as f:
        version = fcecodec.GetFceVersion(f.read(0x2038))
        assert version > 0
        return version

input_fce_version = GetFceVersion(fce_path_input)
print(input_fce_version)

Convert from FCE4M to FCE4, if applicable

In [ ]:
current_fce_version = GetFceVersion(fce_path_input)
if current_fce_version == 5 and fce_version_target < 5:
    pass
    !python "{fcecodec_dir}/scripts/""bfut_PrintFceInfo.py" "{fce_path_input}"
    !python "{fcecodec_dir}/scripts/""bfut_MergeParts (FceM to Fce4, keep version).py" 1 0 1 "{fce_path_input}" "{fce_path_output}"
    !python "{fcecodec_dir}/scripts/""bfut_SaveFceAsFce4.py" "{fce_path_output}" "{fce_path_output}"

    fce_path_input = fce_path_output

Convert from FCE4 to FCE3, if applicable


In [ ]:
current_fce_version = GetFceVersion(fce_path_input)
if current_fce_version == 4 and fce_version_target < 4:
    pass
    !python "{fcecodec_dir}/scripts/""bfut_PrintFceInfo.py" "{fce_path_input}"
    !python "{fcecodec_dir}/scripts/""bfut_MergeParts (Fce4 to Fce3, keep version).py" 1 opaque "{fce_path_input}" "{fce_path_output}"
    !echo "DONE bfut_MergeParts (Fce4 to Fce3, keep version).py"
    !python "{fcecodec_dir}/scripts/""bfut_SortPartsToFce3Order (keep fce version).py" "{fce_path_output}" "{fce_path_output}"
    !python "{fcecodec_dir}/scripts/""bfut_ConvertDummiesToFce3.py" "{fce_path_output}" "{fce_path_output}"
    !python "{fcecodec_dir}/scripts/""bfut_SaveFceAsFce3.py" "{fce_path_output}" "{fce_path_output}"

with open(fce_path_output, "rb") as f:
    assert fcecodec.ValidateFce(f.read()) == 1

Optionally copy colors

In [ ]:
if copy_color:
    pass
    !python "{fcecodec_dir}/scripts/""bfut_CopyCarColors.py" "{fce_path_colorsource}" "{fce_path_output}"

Convert FSH to TGA, if applicable

In [ ]:
if input_fce_version == 5 and fce_version_target < 5 and type(fsh_path_input) is str and len(fsh_path_input) > 0:
    pass
    p = pathlib.Path(fsh_path_input)
    car00tga_input = p.with_suffix(".tga")

    !{fshtool} "{fsh_path_input}"
    BMP = (p.parent / p.stem / "0000").with_suffix(".BMP")
    BMPA = (p.parent / p.stem / "0000-a").with_suffix(".BMP")
    !convert "{BMP}" "{BMPA}" -auto-orient -alpha off -compose CopyOpacity -composite "{car00tga_input}"

Convert TGA alpha channel to target version

In [ ]:
!cp "{car00tga_input}" "{car00tga_output}"

if flip_vertically:
    pass
    !python "{py_scriptsdir}""bfut_TextureRotator/bfut_TextureRotator (flip vertically).py" "{car00tga_output}" "{car00tga_output}"

if fce_version_target == 3:
    if input_fce_version == 4:
        pass
        !python "{py_scriptsdir}""bfut_NfsTgaConverter/bfut_NfsTgaConverter (HSto3).py" "{car00tga_output}" "{car00tga_output}"
    elif input_fce_version == 5:
        pass
        !python "{py_scriptsdir}""bfut_NfsTgaConverter/bfut_NfsTgaConverter (Mto3).py" "{car00tga_output}" "{car00tga_output}"
elif fce_version_target == 4:
    if input_fce_version == 3:
        pass
        !python "{py_scriptsdir}""bfut_NfsTgaConverter/bfut_NfsTgaConverter (3toHS).py" "{car00tga_output}" "{car00tga_output}"
    elif input_fce_version == 5:
        pass
        !python "{py_scriptsdir}""bfut_NfsTgaConverter/bfut_NfsTgaConverter (MtoHS).py" "{car00tga_output}" "{car00tga_output}"
elif fce_version_target == 5:
    if input_fce_version == 3:
        pass
        !python "{py_scriptsdir}""bfut_NfsTgaConverter/bfut_NfsTgaConverter (3toM).py" "{car00tga_output}" "{car00tga_output}"
    elif input_fce_version == 4:
        pass
        !python "{py_scriptsdir}""bfut_NfsTgaConverter/bfut_NfsTgaConverter (HStoM).py" "{car00tga_output}" "{car00tga_output}"

Create VIV archive

In [ ]:
!python "{unvivtool}" e "{viv_source_dir}"

In [ ]:
# Copyright (C) 2023 and later Benjamin Futasz <https://github.com/bfut>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.